# Train데이터 불러오기

각 년도별 DataFrame을 리스트에 append합니다.

리스트 인덱스별로

0: 2017년도 데이터   

1: 2018년도 데이터 

2: 2019년도 데이터 

3: 202년도 데이터 

입니다

In [1]:
import pandas as pd
from glob import glob
from tqdm import tqdm
import warnings

warnings.filterwarnings(action='ignore') 
know_train = [pd.read_csv(path) for path in sorted(glob('./train/*.csv'))]

In [2]:
know_train[0].head() # 2017년도 샘플

,idx,aq1_1,aq1_2,aq2_1,aq2_2,aq3_1,aq3_2,aq4_1,aq4_2,aq5_1,...,bq37,bq38,bq38_1,bq39_1,bq39_2,bq40,bq41_1,bq41_2,bq41_3,knowcode
0,0,3,3,3,3,3,3,4,4,3,...,52,2,실업,1,1,1,4000,,2200,825101
1,1,4,5,4,5,3,4,3,4,3,...,38,4,건축공학,1,1,1,,,2400,140204
2,2,3,4,3,4,3,4,5,6,4,...,50,4,건축공학,1,1,1,4000,,2400,140204
3,3,3,3,3,3,3,5,4,5,4,...,42,4,환경학과,1,1,1,7000,,3500,140601
4,4,4,5,3,4,3,4,4,5,3,...,51,4,건축공학,1,1,1,4000,,2500,140204


# 전처리 

데이터의 빈 셀에 None값이 아닌 ' '처럼 공백으로 들어가있기 때문에 is_null등의 함수로 결측치를 찾아낼 수 없습니다. 

공백이 있는 컬럼은 '0'으로 대체하였습니다.

In [3]:
for df in know_train:
    for col in df.columns:
        df[col].replace(' ', '0', inplace=True)

## 라벨 인코딩

숫자로 변환할 수 있는 컬럼은 라벨 인코딩을 사용하지 않았습니다.

string이나 object컬럼은 라벨인코더를 이용해 변환하였으며 추후 test셋에 사용해야하기 때문에 년도별, 컬럼별로 dictionary를 이용해 저장하였습니다

In [4]:
from sklearn.preprocessing import LabelEncoder
years = ['2017', '2018', '2019', '2020']

year_encoder = {}

for year, df in zip(years, know_train):
    print(year)
    encoders = {}
    
    for col in df.columns:
        if col == 'ID':
            continue
        
        try:
            df[col] = df[col].map(int)
        except:
            encoder = LabelEncoder()
            df[col] = df[col].map(str)
            df[col] = encoder.fit_transform(df[col])
            encoders[col] = encoder
            
            
    year_encoder[year] = encoders

2017
2018
2019
2020


# X, y 구분 및 모델 학습

이번 대회에서 맞춰야 할 값은 knowcode입니다.

ID와 knowcode를 제외한 나머지 feature를 X, knowcode를 정답 y로 두어 모델을 학습하였습니다.

베이스라인에서는 의사결정나무와 랜덤포레스트를 선정하였습니다

In [7]:
train_data = {}
for year, df in zip(years, know_train):
    train_data[year] = {'X': df.iloc[:, 1:-1], # ID제외
                        'y': df.iloc[:, -1]} 

## 의사결정나무

In [8]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier

dt_models = {}

for year in tqdm(years):
    model = DecisionTreeClassifier(random_state=123456)
    model.fit(train_data[year]['X'].iloc[:, :], train_data[year]['y'])
    dt_models[year] = model

100%|██████████| 4/4 [00:10<00:00,  2.59s/it]


## 랜덤포레스트

In [9]:

rf_models = {}

for year in tqdm(years):
    model = RandomForestClassifier(n_estimators=100, random_state=123456, n_jobs=8)
    model.fit(train_data[year]['X'].iloc[:, :], train_data[year]['y'])
    rf_models[year] = model

100%|██████████| 4/4 [00:12<00:00,  3.23s/it]


## Testset 불러오기

마찬가지로 년도별로 DataFrame으로 불러온 후 리스트에 할당합니다.


In [10]:
know_test = [pd.read_csv(path) for path in sorted(glob('./test/*.csv'))]
know_test[0].head() # 2017년도 test 샘플

,idx,aq1_1,aq1_2,aq2_1,aq2_2,aq3_1,aq3_2,aq4_1,aq4_2,aq5_1,...,bq36,bq37,bq38,bq38_1,bq39_1,bq39_2,bq40,bq41_1,bq41_2,bq41_3
0,0,3,4,2,2,3,3,1,,3,...,2,26,3,비서학,1,1,1,3000,,2300
1,1,5,5,3,5,5,5,5,5,4,...,1,57,4,농화학,1,1,1,5500,,2500
2,2,5,5,5,4,5,4,1,,1,...,1,31,4,신문방송,1,1,1,4300,,4000
3,3,4,5,5,6,4,6,3,4,4,...,1,35,6,화학,1,1,1,4100,,3000
4,4,5,6,4,5,4,5,1,,1,...,1,36,4,광고홍보,1,1,1,2800,,2000


Train set과 마찬가지로 평균, '0'으로 대체하였습니다

In [11]:
for df in know_test:
    for col in df.columns:
        df[col].replace(' ', '0', inplace=True)

Train에 사용하였던 라벨인코더를 이용해서 카테고리를 정수로 변환합니다.

만약 Test변수가 Train 라벨인코더의 클래스에 없을경우 -1로 처리합니다

In [12]:
years = ['2017', '2018', '2019', '2020']

for year, df in zip(years, know_test):
    print(year)
    encoders = {}
    
    for col in df.columns:
        
        try:
            df[col] = df[col].map(int)
        except:
            encoder = year_encoder[year][col]
            df[col] = df[col].map(str)
            category_map = {category: idx for idx, category in enumerate(encoder.classes_)}
            df[col] = df[col].apply(lambda x: category_map[x] if x in category_map else -1) # train set에서 보지못한 카테고리변수 -1(UNK) 처리
            
            

2017
2018
2019
2020


### 데이터 처리 및 라벨인코딩 후 

In [13]:
know_train[0]['aq1_1'].value_counts()

3    3116
4    3007
2    1706
5    1072
1     585
Name: aq1_1, dtype: int64

In [14]:
know_test[0].head() # 2017년도 test 샘플

,idx,aq1_1,aq1_2,aq2_1,aq2_2,aq3_1,aq3_2,aq4_1,aq4_2,aq5_1,...,bq36,bq37,bq38,bq38_1,bq39_1,bq39_2,bq40,bq41_1,bq41_2,bq41_3
0,0,3,4,2,2,3,3,1,0,3,...,2,26,3,497,1,1,1,3000,0,2300
1,1,5,5,3,5,5,5,5,5,4,...,1,57,4,287,1,1,1,5500,0,2500
2,2,5,5,5,4,5,4,1,0,1,...,1,31,4,705,1,1,1,4300,0,4000
3,3,4,5,5,6,4,6,3,4,4,...,1,35,6,1423,1,1,1,4100,0,3000
4,4,5,6,4,5,4,5,1,0,1,...,1,36,4,141,1,1,1,2800,0,2000


## 테스트셋 추출 및 학습

ID 값을 제외한 나머지 데이터를 이용하여 모델에 넣어 추론합니다.

In [15]:
test_data = {}
for year, df in zip(years, know_test):
    test_data[year] =  {'X': df.iloc[:,1:]}

### 의사결정나무로 예측

In [16]:
dt_predicts = [] 

for year in tqdm(years):
    pred = dt_models[year].predict(test_data[year]['X'])
    dt_predicts.extend(pred)
    
    

100%|██████████| 4/4 [00:00<00:00, 16.97it/s]


### 랜덤포레스트로 예측

In [17]:
rf_predicts = [] 

for year in tqdm(years):
    pred = rf_models[year].predict(test_data[year]['X'])
    rf_predicts.extend(pred)

100%|██████████| 4/4 [00:04<00:00,  1.25s/it]


# 제출

In [18]:
submission = pd.read_csv('sample_submission.csv') # sample submission 불러오기

In [19]:
submission['knowcode'] = rf_predicts

submission.to_csv('submission.csv', index=False)